In [ ]:
#RBF Network on MNIST (FFN with RBF activation)

In [ ]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
import numpy as np
import matplotlib.pyplot as plt

#for rbf
from scipy import *
from scipy.linalg import norm, pinv

In [ ]:
#importing mnist dataset as a tf dataset
mnist = input_data.read_data_sets("../Data/MNIST/", one_hot=True)

In [ ]:
#fixing some parameters
learning_rate = 0.1
training_epochs = 50
batch_size = 100
display_step = 5

In [ ]:
#defining input tensor (28x28 = 784)
x = tf.placeholder("float", shape=[None, 784])

In [ ]:
#defining final layer tensor
y = tf.placeholder(tf.float32, [None, 10])

In [ ]:
#Gathering evidence

#defining weigths and biases
W  = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [ ]:
#gaussian rbf is simpty squared distance from point in n dimensions. We abstract dimensionality to tensor operations in the numerator.
def rbf(x):
    return tf.exp(-8 *((x)**2)) #letting origin of RBF be 0, beta = 8, and radius be 1

In [ ]:
#computing class probabilities from evidence using softmax:
activation = tf.nn.softmax(tf.matmul(rbf(x),W) + b)

In [ ]:
cross_entropy = y * tf.log(activation)

In [ ]:
#cost function will be calculating the cross-entropy's error (C=−1/n∑x[ylnσ+(1−y)ln(1−σ)], where σ := sigmoid)
cost = tf.reduce_mean(-tf.reduce_sum(cross_entropy, reduction_indices=1))

In [ ]:
#I now have a model, but to train, it needs a learning algorithm
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [ ]:
#defining arrays used for plotting cost function's convergence
avg_set = []
epoch_set=[]

In [ ]:
###--------------All variables have now been written------------------###

In [ ]:
#defining the tf object that will be used to initialize all your variables at once
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    #running the session using context manager
    sess.run(init)
    
    #iterating over each epoch
    for epoch in range(int(training_epochs)):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size) #number of batches

        for i in range(total_batch):
            batch_xs, batch_ys= mnist.train.next_batch(batch_size) #grab a batch

            #fitting the model, using our defined optimizer and passing one batch to the placeholders
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})

            #evaluating cost passing one batch into the placeholders
            avg_cost += sess.run(cost, feed_dict={x: batch_xs, y:batch_ys})/total_batch
            
        #printing epoch and cost each epoch
        if epoch % display_step == 0:
            print("Epoch=", epoch, "; cost=", "{0:.4f}".format(avg_cost))
    
        #saving costs each epoch for plotting
        epoch_set = np.append(epoch_set, epoch)
        avg_set = np.append(avg_set, avg_cost)
    
    #----DONE TRAINING----
    print("Training finished!")
            
    #finding accuracies (correct if highest probability in output is at an index equal to input digit's real number value)
    correct_prediction = tf.equal(tf.argmax(activation, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    print("Model accuracy: ", "{0:.4f}".format(accuracy.eval({x: mnist.test.images, y: mnist.test.labels})))

In [ ]:
#plot cost function convergence as network trains
plt.plot(epoch_set, avg_set)
plt.show()